# find the coresponding sigmoid

In [1]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim
from bm3d import bm3d
from tqdm import tqdm

In [2]:
def calculate_psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:
        return float('inf')
    max_pixel = 1.0
    return 20 * np.log10(max_pixel / np.sqrt(mse))

def calculate_ssim(img1, img2):
    return ssim(img1, img2, data_range=img2.max() - img2.min())

def bm3d_denoise(image_np, sigma_psd):
    denoised_image = bm3d(image_np, sigma_psd=sigma_psd)
    return denoised_image



In [4]:
def process_images(clean_image_path, noisy_image_path, target_size=(2000, 2992)):
    clean_image = Image.open(clean_image_path).convert('L')
    noisy_image = Image.open(noisy_image_path).convert('L')

    clean_image = clean_image.resize(target_size)
    noisy_image = noisy_image.resize(target_size)
    
    clean_np = np.array(clean_image) / 255.0
    noisy_np = np.array(noisy_image) / 255.0

    sigma_values = np.linspace(25, 250, 150) / 255.0
    best_sigma, best_psnr, best_ssim, best_denoised_image, results = find_best_sigma(clean_np, noisy_np, sigma_values)

    best_denoised_image_path = os.path.join(os.path.dirname(clean_image_path), f'best_denoised_{os.path.basename(clean_image_path)}')
    best_denoised_image_pil = Image.fromarray((best_denoised_image * 255).astype(np.uint8))
    best_denoised_image_pil.save(best_denoised_image_path)

    sigma_values = []
    psnr_values = []
    ssim_values = []
    for sigma, psnr_value, ssim_value in results:
        sigma_values.append(sigma)
        psnr_values.append(psnr_value)
        ssim_values.append(ssim_value)

    sigma_values = np.array(sigma_values)
    psnr_values = np.array(psnr_values)
    ssim_values = np.array(ssim_values)

    # Plot PSNR vs Sigma
    plt.figure(figsize=(12, 6))
    plt.plot(sigma_values * 255.0, psnr_values, marker='o', linestyle='-', color='b')
    plt.xlabel('Sigma')
    plt.ylabel('PSNR')
    plt.title(f'PSNR vs Sigma for {os.path.basename(clean_image_path)}')
    plt.grid(True)
    plt.savefig(os.path.join(os.path.dirname(clean_image_path), f'psnr_vs_sigma_{os.path.basename(clean_image_path)}.png'))
    plt.close()

    # Plot SSIM vs Sigma
    plt.figure(figsize=(12, 6))
    plt.plot(sigma_values * 255.0, ssim_values, marker='o', linestyle='-', color='r')
    plt.xlabel('Sigma')
    plt.ylabel('SSIM')
    plt.title(f'SSIM vs Sigma for {os.path.basename(clean_image_path)}')
    plt.grid(True)
    plt.savefig(os.path.join(os.path.dirname(clean_image_path), f'ssim_vs_sigma_{os.path.basename(clean_image_path)}.png'))
    plt.close()

    # Save PSNR and SSIM values to a text file
    results_file_path = os.path.join(os.path.dirname(clean_image_path), f'results_{os.path.basename(clean_image_path)}.txt')
    with open(results_file_path, 'w') as f:
        f.write('Sigma,PSNR,SSIM\n')
        for sigma, psnr_value, ssim_value in results:
            f.write(f"{sigma*255.0:.2f},{psnr_value:.4f},{ssim_value:.4f}\n")

    print(f"Best sigma for {clean_image_path}: {best_sigma}")
    print(f"Best PSNR for {clean_image_path}: {best_psnr}")
    print(f"Best SSIM for {clean_image_path}: {best_ssim}")
    print(f"Best denoised image saved at: {best_denoised_image_path}")
    print(f"Results saved at: {results_file_path}")


    # Plot PSNR vs Sigma
    plt.figure(figsize=(12, 6))
    plt.plot(sigma_values * 255.0, psnr_values, marker='o', linestyle='-', color='b')
    plt.xlabel('Sigma')
    plt.ylabel('PSNR')
    plt.title(f'PSNR vs Sigma for {os.path.basename(clean_image_path)}')
    plt.grid(True)
    plt.savefig(os.path.join(os.path.dirname(clean_image_path), f'psnr_vs_sigma_{os.path.basename(clean_image_path)}.png'))
    plt.close()

    # Plot SSIM vs Sigma
    plt.figure(figsize=(12, 6))
    plt.plot(sigma_values * 255.0, ssim_values, marker='o', linestyle='-', color='r')
    plt.xlabel('Sigma')
    plt.ylabel('SSIM')
    plt.title(f'SSIM vs Sigma for {os.path.basename(clean_image_path)}')
    plt.grid(True)
    plt.savefig(os.path.join(os.path.dirname(clean_image_path), f'ssim_vs_sigma_{os.path.basename(clean_image_path)}.png'))
    plt.close()

    print(f"Best sigma for {clean_image_path}: {best_sigma}")
    print(f"Best PSNR for {clean_image_path}: {best_psnr}")
    print(f"Best SSIM for {clean_image_path}: {best_ssim}")
    print(f"Best denoised image saved at: {best_denoised_image_path}")



In [5]:
def find_best_sigma(clean_np, noisy_np, sigma_values):
    best_sigma = None
    best_psnr = -float('inf')
    best_ssim = -float('inf')
    best_denoised_image = None
    results = []

    with tqdm(sigma_values, desc="Finding best sigma") as pbar:
        for sigma in pbar:
            denoised_image = bm3d_denoise(noisy_np, sigma)
            psnr_value = calculate_psnr(clean_np, denoised_image)
            ssim_value = calculate_ssim(clean_np, denoised_image)

            results.append((sigma, psnr_value, ssim_value))

            if psnr_value > best_psnr and ssim_value > best_ssim:
                best_psnr = psnr_value
                best_ssim = ssim_value
                best_sigma = sigma
                best_denoised_image = denoised_image

# Paths to your image pairs
clean_image_paths = [
    'dataset/temp_source/clean1.jpg',
    'dataset/temp_source/clean2.jpg',
    'dataset/temp_source/clean3.jpg',
    'dataset/temp_source/clean4.jpg'
]
noisy_image_paths = [
    'dataset/temp_source/noisy1.jpg',
    'dataset/temp_source/noisy2.jpg',
    'dataset/temp_source/noisy3.jpg',
    'dataset/temp_source/noisy4.jpg'
]

# Process each image pair with a progress bar
for clean_image_path, noisy_image_path in zip(clean_image_paths, noisy_image_paths):
    print(clean_image_paths)
    process_images(clean_image_path, noisy_image_path)

['dataset/temp_source/clean1.jpg', 'dataset/temp_source/clean2.jpg', 'dataset/temp_source/clean3.jpg', 'dataset/temp_source/clean4.jpg']


Finding best sigma: 100%|██████████| 150/150 [1:56:43<00:00, 46.69s/it]


TypeError: cannot unpack non-iterable NoneType object